In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!unzip /content/drive/MyDrive/fake-news.zip

Archive:  /content/drive/MyDrive/fake-news.zip
  inflating: submit.csv              
  inflating: test.csv                
  inflating: train.csv               


In [7]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [10]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

###**Import Data**

In [11]:
news_dataset=pd.read_csv('/content/train.csv')
news_dataset.head(5)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


###**Counting number of missing values**


In [12]:
news_dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

###**Replacing missing values with empty strings**

In [13]:
news_dataset=news_dataset.fillna('')

###**Merging the author and titles columns**

In [14]:
news_dataset['content']=news_dataset['author']+' '+news_dataset['title']

In [15]:
news_dataset.head(1)

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...


###**Seperating the data and label**

In [16]:
X=news_dataset.drop(columns='label',axis=1)
Y=news_dataset['label']
print(X)
print(Y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

###**Stemming**:
it's the process of removing the affixes to get the root of the word , we do it before vectorizing the data to get numerical features will be using the PorterStemmer function

In [17]:
port_stem=PorterStemmer()

In [18]:
#
def stemming(content):
  stemmed_content=re.sub('[^a-zA-Z]',' ', content) # remove numbers and special characters and keep words alphabitical
  stemmed_content=stemmed_content.lower() # transform the word we got to lower case
  stemmed_content=stemmed_content.split() # get list of word from the paragraph
  stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')] # get the stem of the words if they are not stop words
  stemmed_content=' '.join(stemmed_content) #keeping the stemmed words
  return stemmed_content

In [19]:
#apply steming on the content column
news_dataset['content']=news_dataset['content'].apply(stemming)

In [20]:
news_dataset['content'].head(1)

0    darrel lucu hous dem aid even see comey letter...
Name: content, dtype: object

In [21]:
X=news_dataset['content'].values
Y=news_dataset['label'].values

###**Vectorizing the textuel data**

In [22]:
vectorizer=TfidfVectorizer()
vectorizer.fit(X)

X=vectorizer.transform(X) #this will find the most rpeated words and give valaue acording to it for each row to feed to our machine learning model

In [23]:
print(X)

  (0, 15686)	0.28485063562728646
  (0, 13473)	0.2565896679337957
  (0, 8909)	0.3635963806326075
  (0, 8630)	0.29212514087043684
  (0, 7692)	0.24785219520671603
  (0, 7005)	0.21874169089359144
  (0, 4973)	0.233316966909351
  (0, 3792)	0.2705332480845492
  (0, 3600)	0.3598939188262559
  (0, 2959)	0.2468450128533713
  (0, 2483)	0.3676519686797209
  (0, 267)	0.27010124977708766
  (1, 16799)	0.30071745655510157
  (1, 6816)	0.1904660198296849
  (1, 5503)	0.7143299355715573
  (1, 3568)	0.26373768806048464
  (1, 2813)	0.19094574062359204
  (1, 2223)	0.3827320386859759
  (1, 1894)	0.15521974226349364
  (1, 1497)	0.2939891562094648
  (2, 15611)	0.41544962664721613
  (2, 9620)	0.49351492943649944
  (2, 5968)	0.3474613386728292
  (2, 5389)	0.3866530551182615
  (2, 3103)	0.46097489583229645
  :	:
  (20797, 13122)	0.2482526352197606
  (20797, 12344)	0.27263457663336677
  (20797, 12138)	0.24778257724396507
  (20797, 10306)	0.08038079000566466
  (20797, 9588)	0.174553480255222
  (20797, 9518)	0.295420

###**Spliting the dataset to train test**

In [24]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2)

###**Training the logistic regression model**


In [25]:
model= LogisticRegression()

In [27]:
model.fit(X_train,Y_train)

LogisticRegression()

###**Evaluation**

In [31]:
#accuracy_score on training data
X_train_pred=model.predict(X_train)
training_data_acc=accuracy_score(X_train_pred,Y_train)

In [32]:
print(f'the accuracy score for training {training_data_acc}' )

the accuracy score for training 0.9865985576923076


In [33]:
#accuracy_score on test data
X_test_pred=model.predict(X_test)
test_data_acc=accuracy_score(X_test_pred,Y_test)

In [34]:
print(f'the accuracy score for test {test_data_acc}' )

the accuracy score for test 0.9790865384615385


###**Predictive System**

In [36]:
X_news=X_test[0] #taking the first news as an exp
prediction=model.predict(X_news)
print(prediction)
if prediction[0]==0 :
  print('real news')
else:
  print("fake news")

[1]
fake news
